# Lending Club - Loan repayment prediction (Logistic Regression)


Lending club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors seeing to lend money and make a return.

The borrower fills out a comprehensive application, providing their past financial history, the reason for the loan and more.Lending Club evaluates the borrowers credit score using past historical data and assigns an interest rate to the borrower. The interest rate ranges from 5.32% to 30.99%. Higher the interest rate greater is the risk. The borrowers are graded according to the interest rate they were assigned. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.
    
The borrowers make repayments over 36 to 60 months. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.
se 
With the mindset of a conservative investor we want to invest in only those loans that have a good chance of being paid off on time.To do that we need to understad the features of the dataset and then build machine learning models that reliably predict whether a loan will be paid off or not.
   

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs

%matplotlib inline

In [3]:
# We are reading data of all approved loans between 2007 and 2011.
loans_2007=pd.read_csv('loans_2007.csv')

C:\Users\hai\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-13,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [5]:
len(loans_2007.columns)

52

In [6]:
loans_2007.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

As there are 52 columns, analysing all the columns will be time consuming and hence we will split the columns into 3 sets of 18 columns each and decide further on to choose only a few among them.

# Data cleaning

In [7]:
# In the first set,the following columns either leak info from future , dont affect the ability to pay back , formatted poorly , 
# contain redundant info , require more data for processing. Hence we can drop these columns.

loans_2007=loans_2007.drop(['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d'],axis=1)

In [8]:
# The listed columns leak data from future or contain redundant info.We will drop these as well.

loans_2007=loans_2007.drop(['zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp'],axis=1)

In [9]:
# The last set also contians columns which leak data from future.

loans_2007 = loans_2007.drop(['total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d',
'last_pymnt_amnt'],axis=1)

Now the total number of columns have been reduced to 32.

The loan_status columns is the one which will be the target columns as it is the only one which describes if a loan was paid off on time or not.

In [10]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

Only the 'Fully Paid' and 'Charged Off' values describe the final outcome.Since we're interested in predicting whether a loan will be paid off or not we treat the problem as a binary classification.
'Fully Paid' is replaced with '1' and 'Charged Off' is replaced with '0'.

In [11]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]

In [12]:
loans_2007['loan_status'] = loans_2007['loan_status'].replace('Fully Paid',1)

In [13]:
loans_2007['loan_status'] = loans_2007['loan_status'].replace('Charged Off',0)

We will also drop collumns that have only 1 unique value in them as that column will not add any information to the loan application.

In [14]:
drop_columns=[]
for col in loans_2007.columns:
    check=loans_2007[col].dropna().unique()
    vals=len(check)
    if vals==1:
        drop_columns.append(col)
        
    

In [15]:
drop_columns

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [16]:
loans_2007 = loans_2007.drop(drop_columns,axis=1)

In [17]:
len(loans_2007.columns)

23

We have now further reduced the number of columns from 32 to 23.

In [18]:
# Exporting the refined data to a new csv file.
filtered_loans=loans_2007.to_csv('filtered_loans_2007.csv')

 # Feature preparation

In [19]:
import pandas as pd
loans = pd.read_csv('filtered_loans_2007.csv')
null_counts=loans.isnull().sum()

In [20]:
null_counts

Unnamed: 0                 0
loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

In [21]:
len(loans)

38770

We will remove columns which have more than 1% of the total rows as missing values. And drop rows where there are null values for other columns which have <1% of the values missing.

In [22]:
loans=loans.drop('pub_rec_bankruptcies',axis=1)

In [23]:
loans=loans.dropna(axis=0)

In [24]:
loans.dtypes.value_counts()

object     11
float64    10
int64       2
dtype: int64

The object columns that contain text need to be converted to numerical data types as the final input data needs only numeric values.

In [25]:
# selecting data types of all columns where the 'dtype' is 'object' so that they can be either dropped or reformatted.

object_cols=loans.select_dtypes(include=['object'])
object_cols.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-85,83.70%,Jun-16
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-99,9.40%,Sep-13
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-01,98.50%,Jun-16
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-96,21%,Apr-16
4,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-04,28.30%,Jan-16


In the above data we notice that there are 'numerical with string' values. Also 'categorical' values present. We further need to re-engineer those values.

In [26]:
# Removing the '%' symbol and converting to float values in the 'interest rate' and 'revolving utilization' columns.

loans['int_rate']=loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util']=loans['revol_util'].str.rstrip('%').astype('float')

In [27]:
mapping_dict= {"10+ years":10,"9 years":9,"8 years":8,"7 years":7,"6 years":6,"5 years":5,"4 years":4,
                            "3 years":3,"2 years":2,"1 year":1,"<1 year":0,"n/a":0}

In [28]:
# Converting all 'string' values to 'integer' values in the 'emloyee tenure' column.

loans['emp_length']=loans['emp_length'].replace('10+ years',10)
loans['emp_length']=loans['emp_length'].replace('9 years',9)
loans['emp_length']=loans['emp_length'].replace('8 years',8)
loans['emp_length']=loans['emp_length'].replace('7 years',7)
loans['emp_length']=loans['emp_length'].replace('6 years',6)
loans['emp_length']=loans['emp_length'].replace('5 years',5)
loans['emp_length']=loans['emp_length'].replace('4 years',4)
loans['emp_length']=loans['emp_length'].replace('3 years',3)
loans['emp_length']=loans['emp_length'].replace('2 years',2)
loans['emp_length']=loans['emp_length'].replace('1 year',1)
loans['emp_length']=loans['emp_length'].replace('< 1 year',0)

In [29]:
# Verifying the values in 'employee tenure' column.

loans['emp_length']

0        10
1         0
2        10
3        10
4         3
5         8
6         9
7         4
8         0
9         5
10       10
11        0
12        3
13        3
14        0
15        4
16       10
17        1
18        6
19        3
20       10
21       10
22        5
23        1
24        2
25        9
26        2
27       10
28        0
29        7
         ..
38740     9
38741     6
38742     0
38743     3
38744     1
38745     1
38746     0
38747     8
38748     7
38749     2
38750     0
38751     1
38752     4
38753     0
38754     7
38755     1
38756     2
38757     0
38758     8
38759     0
38760     2
38761     4
38762     0
38763     1
38764     5
38765     4
38766     3
38767     0
38768     0
38769     0
Name: emp_length, Length: 37675, dtype: int64

In [30]:
# There are other redundant columns which we can drop. These can be deemed to add no value for our prediction.

loans=loans.drop(['last_credit_pull_d','addr_state','title','earliest_cr_line'],axis=1)

In [31]:
loans.head()

,Unnamed: 0,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
4,5,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


In [32]:
loans.columns

Index(['Unnamed: 0', 'loan_amnt', 'term', 'int_rate', 'installment',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'loan_status', 'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc'],
      dtype='object')

In [33]:

cat_columns=['home_ownership','verification_status','purpose','term']
dummy_df=pd.get_dummies(loans[['home_ownership','verification_status','purpose','term']])
loans=pd.concat([loans,dummy_df],axis=1)
loans=loans.drop(cat_columns,axis=1)

In [34]:
loans.head()

,Unnamed: 0,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
1,1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,...,0,0,0,0,0,0,0,0,0,1
2,2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,...,0,0,0,1,0,0,0,0,1,0
4,5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,...,0,0,0,0,0,0,0,1,1,0


In [35]:
cleaned_loans_2007=loans.to_csv('cleaned_loans_2007.csv')


In [36]:
cleaned_loans=pd.read_csv('cleaned_loans_2007.csv')

In [37]:
cleaned_loans.shape

(37675, 40)

In [38]:
cleaned_loans.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'loan_amnt', 'int_rate', 'installment',
       'emp_length', 'annual_inc', 'loan_status', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months'],
      dtype='object')

In [39]:
cols=['loan_amnt','int_rate','emp_length','annual_inc','dti','delinq_2yrs','total_acc','purpose_car',
              'purpose_credit_card','purpose_debt_consolidation','purpose_educational','purpose_home_improvement',
              'purpose_house','purpose_medical','purpose_small_business']

In [40]:
train_cols=cleaned_loans[cols]
test_col=cleaned_loans['loan_status']

In [41]:
train_cols.head()

,loan_amnt,int_rate,emp_length,annual_inc,dti,delinq_2yrs,total_acc,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_medical,purpose_small_business
0,5000.0,10.65,10,24000.0,27.65,0.0,9.0,0,1,0,0,0,0,0,0
1,2500.0,15.27,0,30000.0,1.00,0.0,4.0,1,0,0,0,0,0,0,0
2,2400.0,15.96,10,12252.0,8.72,0.0,10.0,0,0,0,0,0,0,0,1
3,10000.0,13.49,10,49200.0,20.00,0.0,37.0,0,0,0,0,0,0,0,0
4,5000.0,7.90,3,36000.0,11.20,0.0,12.0,0,0,0,0,0,0,0,0


In [42]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression()
logr.fit(cleaned_loans[['int_rate']],cleaned_loans['loan_status'])
train_predictions=logr.predict(cleaned_loans[['int_rate']])
cleaned_loans['predicted_status']=train_predictions

In [43]:
cleaned_loans['predicted_status'].value_counts()

1    37661
0       14
Name: predicted_status, dtype: int64

In [44]:
# Matching the 'actual_status' with the 'predicted_status'.

matches= cleaned_loans['loan_status'] == cleaned_loans['predicted_status']
correct_predictions=cleaned_loans[matches]

In [45]:
# Computing the accuracy of our predictions.

accuracy = len(correct_predictions) / len(cleaned_loans)

print(accuracy)

0.8568015925680159


The above number represents the accuracy of our predicted values from the actual values. Hence we can state that our model is 85% accurate. 

Though there might an over fitting in our results we ca try to optimise it by training with different datasets and vary the feature valriables or even try optimising techniques.